In [ ]:
from plotly import graph_objects as go

In [ ]:
import torch
import torchvision
from torch import nn
# print(torch.__version__)
# print(torchvision.__version__)

In [ ]:
from torchvision import transforms
from modularized.engine import vggTrainingInfer, build_engine
from modularized.data_processes import get_data, create_dataloaders

In [ ]:
import numpy as np
from PIL import Image
# example image:
np.asarray(Image.open('data/pizza_steak_sushi/test/pizza/194643.jpg')).shape

In [ ]:
import matplotlib.pyplot as plt
from torchinfo import summary

In [ ]:
device='cpu'

In [ ]:
train_dir, test_dir=get_data(
    data_path='data/pizza_steak_sushi',
    data_url='https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',
    unzip=True,
)


In [ ]:
# auto creation of transformations
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transforms=weights.transforms()

# automatic transformations
train_dataloader, test_dataloader, class_names, class_dict = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    img_size=(
        auto_transforms.crop_size[0], auto_transforms.crop_size[0]
    ),
    transformations=auto_transforms,
    batch_size=16,
    num_workers=4,
)

In [ ]:
model=torchvision.models.efficientnet_b0(pretrained=True).to(device)

In [ ]:
# getting a summary of our model with torchinfo.summary()
summary(
    model, 
    input_size=(1,3,224,224),
    col_names=['input_size', 'output_size', 'num_params', 'trainable'],
    col_width=20,
    row_settings=['var_names']
)

In [ ]:
# freezing the base model and chaging the output layer to suit the needs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

for param in model.features.parameters():
    param.requires_grad=False

# Update the classifier head of our model to suit the problem
model.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(
        in_features=1280,
        out_features=len(class_names)
    )
).to(device)

In [ ]:
# setting up loss function and optimizer
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(
  model.parameters(),
  lr=5e-4
)

# create engine and train
print('performing training of the model')
model, results=build_engine(
  model=model,
  loss_fn=loss_fn,
  optimizer=optimizer,
  train_dataloader=train_dataloader,
  test_dataloader=test_dataloader,
  epochs=20,
  device=device
)

In [ ]:
# plot loss curves:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(10),
        y=results['train_loss'],
        name='train_loss'
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(10),
        y=results['eval_loss'],
        name='test_loss'
    )
)
fig.update_layout(dict(
    title='Loss Plots',
    width=500,
    height=500
))
fig.show()

# plot loss curves:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(10),
        y=results['train_metric'],
        name='train_acc'
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(10),
        y=results['eval_metric'],
        name='test_acc'
    )
)
fig.update_layout(dict(
    title='Accuracy Plots',
    width=500,
    height=500
))
fig.show()

In [ ]:
from typing import List, Tuple
from PIL import Image
from torchvision import transforms

def pred_and_plot_image(
    model,
    image_path,
    class_names,
    image_size,
    transform=None,
    device='cpu',
):
    img=Image.open(image_path)

    if transform is None:
        transform=transforms.Compose(
            [
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                )
            ]
        )
    
    model.to(device)
    model.eval()
    with torch.inference_mode():
        transformed_image=transform(img).unsqueeze(dim=0)
        target_image_pred=model(transformed_image.to(device))
    
    target_image_pred_probs=torch.softmax(target_image_pred, dim=1)
    target_image_pred_label=torch.argmax(target_image_pred_probs, dim=1).item()
    target_image_pred_probs=target_image_pred_probs.numpy()

    plt.figure()
    plt.imshow(img)
    plt.title(f'Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs[0][target_image_pred_label]*100:.3f}%')

In [ ]:
pred_and_plot_image(
    model,
    image_path='data/pizza_steak_sushi/eval/04-pizza-dad.jpeg',
    class_names=class_names,
    image_size=(224,224),
    transform=auto_transforms,
    device='cpu',
)